# Template

In [ ]:
import numpy as np
import time
from datetime import datetime, timedelta
from IPython.display import clear_output
from neuroplatform import StimShape, StimParam , IntanSofware, Trigger, Database, StimPolarity, Experiment

token = "XSALK6J9C4" 
exp = Experiment(token)
print(f'Electrodes: \{exp.electrodes\}') # Electrodes that you can used

stim_params = []
for i in range(8):
     # Creating an instance of StimParam to hold the stimulation parameters
    stim_param = StimParam()
    
    # Assigning the first electrode from the 'exp.electrodes' list to be the target for stimulation
    stim_param.index = exp.electrodes[0] # Always on the same electrode
    
    # 'trigger_key' is likely a unique identifier for the stimulation; it's set to the value of 'i' 
    stim_param.trigger_key = i
    
    # 'trigger_delay' is set to 0, indicating that the stimulation should begin immediately upon triggering
    stim_param.trigger_delay = 0
    
    # Setting the number of pulses in the stimulation train to 40
    stim_param.nb_pulse = 40
    
    # Defining the period of the pulse train; here set to 5,000,000 microseconds (5 seconds) between each pulse
    stim_param.pulse_train_period = 500000 # 5s between each pulse -> 40*5s = 200s total for the entire pulse train
    
    # Setting the refractory period post-stimulation to 4.9996 seconds (4999600 microseconds) before a new stimulation can begin
    stim_param.post_stim_ref_period = 4999600.0 # After the 200s pulse train, wait for 4.9996s
    
    # Specifying the shape of the stimulation to be biphasic, meaning each pulse consists of two phases of opposite polarity
    stim_param.stim_shape = StimShape.Biphasic
    
    # Setting the polarity of the first phase to be positive
    stim_param.polarity = StimPolarity.PositiveFirst
    
    # Defining the duration of the first phase of each biphasic pulse to be 200 microseconds
    stim_param.phase_duration1 = 200.0
    
    # Defining the duration of the second phase of each biphasic pulse to also be 200 microseconds
    stim_param.phase_duration2 = 200.0
    
    # Setting the amplitude of the first phase of the biphasic pulse to 8
    stim_param.phase_amplitude1 = 8.0 # [1-8 uA]
    
    # Setting the amplitude of the second phase of the biphasic pulse to 8, matching the first phase
    stim_param.phase_amplitude2 = 8.0 # [1-8 uA] , Equal amplitude for both phases of the biphasic pulse
    
    # Enabling the amplifier settling feature which allows the amplifier to stabilize before delivering the stimulus
    stim_param.enable_amp_settle = True
    
    # There is no pre-stimulus amplifier settling time set, as this value is 0.0
    stim_param.pre_stim_amp_settle = 0.0
    
    # Setting the post-stimulus amplifier settling time to 1 millisecond (1000 microseconds)
    stim_param.post_stim_amp_settle = 1000.0
    
    # Enabling charge recovery, which helps to balance the charge injected into the tissue and is important for safety
    stim_param.enable_charge_recovery = True
    
    # Setting the time for the charge recovery to begin immediately after the stimulus (0 milliseconds delay)
    stim_param.post_charge_recovery_on = 0.0
    
    # Setting the duration of the charge recovery to 100 microseconds
    stim_param.post_charge_recovery_off = 100.0
    
    # There is no delay between the phases of the biphasic pulse, as indicated by an interphase delay of 0.0
    stim_param.interphase_delay = 0.0 
    stim_params.append(stim_param)

intan = IntanSofware()
trigger_gen = Trigger()

try:
    if exp.start(): # Signal the start of an experiment to all users
        # Measure impedance
        intan.impedance()

        # Disable Variation STD (keep a fixed threshold)
        intan.var_threshold(False)

        # Send stim parameter
        intan.send_stimparam(stim_params)

        trigger0 = np.zeros(16, dtype=np.uint8)    

        for i in range(8):
            trigger0[i] = 1
            time.sleep(300) # Sleep 300s before stim
            trigger_gen.send(trigger0)
            time.sleep(300) # Sleep 300s after sending stim
            trigger0[i]=0

        # Disable all stims
        for stim in stim_params:
            stim.enable = False
        intan.send_stimparam(stim_params)

finally:
    # Close the connection to trigger generator
    trigger_gen.close()
    # Enable variation threshold again
    intan.var_threshold(True)
    # Close the connection to intan software
    intan.close()
    # Signal the end of an experiment to all users
    exp.stop()

# Stimulation

In [ ]:
import numpy as np
import time
from neuroplatform import StimShape, StimParam, IntanSoftware, Trigger, Experiment, StimPolarity
from fbm import FBM

def generate_oscillatory_sequence(length, frequency, amplitude=1):
    t = np.linspace(0, 1, length)
    oscillatory_sequence = amplitude * np.sin(2 * np.pi * frequency * t)
    return oscillatory_sequence

token = "XSALK6J9C4"
exp = Experiment(token)
print(f'Electrodes: {exp.electrodes}')

def generate_fbmn_sequence(length, min_val, max_val, hurst):
    f = FBM(n=length-1, hurst=hurst, length=1, method='daviesharte')
    fbmn = f.fbm()
    fbmn_normalized = min_val + (max_val - min_val) * (fbmn - np.min(fbmn)) / (np.max(fbmn) - np.min(fbmn))
    return fbmn_normalized

stim_params = []
for i in range(8):
    fractal_amplitude = generate_fbmn_sequence(40, 1, 8, 0.7)
    fractal_duration = generate_fbmn_sequence(40, 100, 200, 0.7)
    oscillatory_amplitude = generate_oscillatory_sequence(40, frequency=0.1, amplitude=1)  # Adding oscillations

    stim_param = StimParam()
    stim_param.index = exp.electrodes[0]
    stim_param.trigger_key = i
    stim_param.trigger_delay = 0
    stim_param.nb_pulse = 40
    stim_param.pulse_train_period = 5000000
    stim_param.post_stim_ref_period = 4999600.0
    stim_param.stim_shape = StimShape.Biphasic
    stim_param.polarity = StimPolarity.PositiveFirst
    stim_param.phase_duration1 = fractal_duration[i]
    stim_param.phase_duration2 = fractal_duration[i]
    stim_param.phase_amplitude1 = fractal_amplitude[i] + oscillatory_amplitude[i]
    stim_param.phase_amplitude2 = fractal_amplitude[i] + oscillatory_amplitude[i]
    stim_param.enable_amp_settle = True
    stim_param.pre_stim_amp_settle = 0.0
    stim_param.post_stim_amp_settle = 1000.0
    stim_param.enable_charge_recovery = True
    stim_param.post_charge_recovery_on = 0.0
    stim_param.post_charge_recovery_off = 100.0
    stim_param.interphase_delay = 0.0
    stim_params.append(stim_param)

intan = IntanSoftware()
trigger_gen = Trigger()

try:
    if exp.start():
        intan.impedance()
        intan.var_threshold(False)
        intan.send_stimparam(stim_params)
        trigger0 = np.zeros(16, dtype=np.uint8)
        for i in range(8):
            trigger0[i] = 1
            time.sleep(300)
            trigger_gen.send(trigger0)
            time.sleep(300)
            trigger0[i] = 0
        for stim in stim_params:
            stim.enable = False
        intan.send_stimparam(stim_params)
finally:
    trigger_gen.close()
    intan.var_threshold(True)
    intan.close()
    exp.stop()

# Visualisation

In [3]:
import numpy as np
from fbm import FBM

def generate_oscillatory_sequence(length, frequency, amplitude=1):
    t = np.linspace(0, 1, length)
    oscillatory_sequence = amplitude * np.sin(2 * np.pi * frequency * t)
    return oscillatory_sequence

def generate_fbmn_sequence(length, min_val, max_val, hurst):
    f = FBM(n=length-1, hurst=hurst, length=1, method='daviesharte')
    fbmn = f.fbm()
    fbmn_normalized = min_val + (max_val - min_val) * (fbmn - np.min(fbmn)) / (np.max(fbmn) - np.min(fbmn))
    return fbmn_normalized

def create_stim_signal(amplitude_sequence, duration_sequence):
    signal = []
    for amplitude, duration in zip(amplitude_sequence, duration_sequence):
        # Each pulse consists of a positive phase followed by a negative phase
        pulse = [amplitude] * int(duration) + [-amplitude] * int(duration)
        signal.extend(pulse)
    return signal

# Initialize arrays for the parameters
phase_durations = []
phase_amplitudes = []

# Generate the sequences for each parameter
for i in range(8):
    fractal_amplitude = generate_fbmn_sequence(40, 1, 8, 0.7)
    fractal_duration = generate_fbmn_sequence(40, 100, 200, 0.7)
    oscillatory_amplitude = generate_oscillatory_sequence(40, frequency=0.1, amplitude=1)
    
    # Combine fractal and oscillatory sequences for amplitudes
    combined_amplitude = fractal_amplitude + oscillatory_amplitude
    
    # Append generated values to the lists
    phase_durations.extend(fractal_duration)
    phase_amplitudes
extend(combined_amplitude)
# Create the stimulation signals

stim_signal_1 = create_stim_signal(phase_amplitudes, phase_durations)
stim_signal_2 = create_stim_signal(phase_amplitudes, phase_durations) # Same as signal 1 for biphasic
# The time vector generation will depend on the actual timing of your stimulations
# For example purposes, we're assuming each phase_duration unit is equivalent to 1ms

time_vector = np.arange(0, len(stim_signal_1))

NameError: name 'phase1_time' is not defined